In [73]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
import time
import sys, os

In [187]:
def get_percentile(x, t='latest',val=0):
    if t=='latest':
        x=x.dropna().values
        val=x[-1]
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    else:
        return  np.percentile(np.sort(x),val)

In [188]:
# RSI, %time above RSI
def RSI(series, period=14):
 #print(len(series))
 delta = series.diff().dropna()
 u = delta * 0
 d = u.copy()
 u[delta > 0] = delta[delta > 0]
 d[delta < 0] = -delta[delta < 0]
 u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
 u = u.drop(u.index[:(period-1)])
 d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
 d = d.drop(d.index[:(period-1)])
 #rs = pd.stats.moments.ewma(u, com=period-1, adjust=False) / pd.stats.moments.ewma(d, com=period-1, adjust=False)
 rs = pd.Series.ewm(u, com=period-1).mean()/pd.Series.ewm(d, com=period-1).mean()
 rsi = 100 - 100 / (1 + rs)
 return np.round(rsi.values[-1],2),get_percentile(rsi)#np.round(100*np.sum(np.where(rsi>rsi.values[-1],1,0))/len(rsi),2)

In [189]:
def consecutive(x,lag=4):
    c=0
    for i in x[-lag:][::-1]:
        if i>=0:
            c+=1
        else:
            return c
    return c    

def consecutive_increases(x):
    c=0
    latest=x.values[-1]
    for i in x[::-1]:
        if i==latest:
            continue
        if latest>i:
            c+=1
            latest=i
        else:
            return c
    return c    

In [190]:
# Consecutive increases, hist percentile, hist change mean 4 week, slope of 4 week macd
def macd_calcs(x):
    macd=x.ewm(span=12,min_periods=0,adjust=True,ignore_na=True).mean()-x.ewm(span=26,min_periods=0,adjust=True,ignore_na=True).mean()
    signal=macd.ewm(span=9,min_periods=0,adjust=True,ignore_na=True).mean()
    macd_hist=macd-signal
    return consecutive_increases(macd_hist), get_percentile(macd_hist),(macd_hist/macd_hist.shift(1)-1).rolling(4).mean().values[-1:][0],np.polyfit(x=[1,2,3,4],y=macd.values[-4:],deg=1,full=False)[0]

In [191]:
def get_intraday(ticker):
    url='https://cloud.iexapis.com/stable/stock/{}/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartLast=10&chartIEXWhenNull=True&chartInterval=10'
    d={'chartIEXOnly': False}
    response = requests.get(
        url.format(ticker),
       params=d
        )
    for i in response.json()[::-1]:
        if ((i['close'] is None)==False):
            return ([i['open'],i['high'],i['low'],i['close'],i['close']])
    url='https://cloud.iexapis.com/stable/stock/{}/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartLast=10&chartIEXWhenNull=True&chartInterval=10'
    d={'chartIEXOnly': True}
    response = requests.get(
        url.format(ticker),
       params=d
        )
    for i in response.json()[::-1]:
        if ((i['marketClose'] is None)==False):
            return([i['marketOpen'],i['marketHigh'],i['marketLow'],i['marketClose'],i['marketClose']])

In [241]:
#
holdings='SPY,MSFT,SVMK,CHWY,EA,OLLI,SHOP,NEWR,DKS,MBUU,ADBE,CLDR,QQQ,TSLA,ESTC,CLX,MELI,NYT,CWH,DKNG,ARKF,THO,TTD,FTNT,WK,NET,SNAP,DDOG,W'


In [242]:
holdings=holdings.split(',')

In [243]:
#holdings=['GDX','RHHBY','ETSY','ECL','OLLI','BLK','SPY','SPXL','AZN','CDNS','CWH','BYND']
#holdings=['RH']

In [244]:
res=[[],[],[],[],[],[],[],[],[],[],[],[],[]]

In [245]:
c=1
for ticker in holdings:
    print(ticker)
    time.sleep(1)
    c+=1
    if c==4:
        c=1
        time.sleep(60)
    d={'function': 'TIME_SERIES_DAILY_ADJUSTED','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'outputsize':'full'}
    url='https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey=YEUC2DL0F7KC43PB&outputsize=full'.format(ticker)
    response = requests.get(
        url
#        params=d,
        )
    #print(response.text)
    #print(url)
    try:
        spy=[[],[],[],[],[],[],[]]
        #print(response.text)
        for key, val in response.json()['Time Series (Daily)'].items():
            spy[0].append(float(val['1. open']))
            spy[1].append(float(val['2. high']))
            spy[2].append(float(val['3. low']))
            spy[3].append(float(val['4. close']))
            spy[4].append(float(val['5. adjusted close']))
            #spy[5].append(float(val['6. volume']))
            #spy[6].append(float(val['7. dividend amount']))
        t=list(response.json()['Time Series (Daily)'].keys())
        
        try:
            if (pd.to_datetime(t[0]).date()==pd.Timestamp.today().date())==False:
                x=get_intraday(ticker)
                spy[0].insert(0,float(x[0]))
                spy[1].insert(0,float(x[1]))
                spy[2].insert(0,float(x[2]))
                spy[3].insert(0,float(x[3]))
                spy[4].insert(0,float(x[4]))
                t.insert(0,str(pd.Timestamp.today().date()))

                #spy[0].append(float(t.insert(0,response2.json()[0]['date'])))
        except:
            print(ticker,'IEX')

        


        spy=pd.DataFrame({'Date':t,
                     'open':spy[0],
                      'high':spy[1],
                      'low':spy[2],
                      'close':spy[3],
                      'adj_close':spy[4]#,
                    #  'volume':spy[5],
                    #  'dividend':spy[6],
                     })
        spy=spy.astype({'Date': 'datetime64'})
        spy=spy.sort_values(by='Date')
        spy['highest_high']=spy.high.rolling(22).max()
        spy['lowest_low']=spy.low.rolling(22).min()
        spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
        spy['k']=spy.k.rolling(3).mean()
        spy['k_change'] = spy.k/spy.k.shift(3)-1
        res[0].append(ticker)
        res[1].append(spy.iloc[-1]['Date']) 
        res[2].append(spy.iloc[-1]['k']) 
        res[3].append(spy.iloc[-1]['k_change'])
        res[4].append(np.nanpercentile(spy.k_change,q=10))
        res[5].append(get_percentile(spy.k_change))
        temp_res=RSI(spy.adj_close)
        res[6].append(temp_res[0])
        res[7].append(temp_res[1])
        temp_res=macd_calcs(spy.adj_close)
        res[8].append(temp_res[0])
        res[9].append(temp_res[1])
        res[10].append(spy.adj_close.rolling(90).std().values[-1])
        res[11].append((spy.adj_close/spy.adj_close.shift(5)-1).rolling(4).mean().values[-1])
        res[12].append((spy.adj_close/spy.adj_close.shift(22)-1).values[-1])
        time.sleep(1)
        
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(ticker,exc_type, exc_tb.tb_lineno)  


SPY
MSFT
SVMK
CHWY
EA
OLLI
SHOP
NEWR
DKS
MBUU
ADBE
CLDR
CLDR <class 'KeyError'> 20
QQQ
TSLA
ESTC
CLX
MELI
NYT
NYT <class 'KeyError'> 20
CWH
DKNG
ARKF
THO
TTD
FTNT
WK
NET
SNAP
DDOG
W


In [246]:
res=pd.DataFrame({'ticker':res[0], 
            'date':res[1],
             'k':res[2],
             'k_change':res[3],
             'k_10_percentile':res[4],
             'k_change_percentile':res[5],
             'RSI':res[6],
             'RSI_percentile':res[7],
             'hist_cons':res[8],    
             'hist_percentile':res[9],
             '90dayStd':res[10],
             'ret1wk_mean':res[11],
             '22dayret':res[12]    
             }
            )

In [247]:
res.sort_values(by='k_change_percentile',ascending=True)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret
17,DKNG,2020-06-25,0.453561,-0.390069,-0.274916,7.28,47.14,12.39,0,0.00,10.449977,-0.092349,0.047469
0,SPY,2020-06-25,0.386321,-0.325179,-0.453047,14.86,48.41,31.22,0,0.83,24.742986,-0.006729,0.016863
9,MBUU,2020-06-25,0.397761,-0.294905,-0.429637,17.28,51.13,45.37,0,2.42,10.871005,-0.024467,0.040916
5,OLLI,2020-06-25,0.701296,-0.242750,-0.399259,18.52,62.88,76.54,0,8.03,20.253678,0.029941,0.110511
16,CWH,2020-06-25,0.695901,-0.248184,-0.510967,20.79,61.10,77.74,0,10.27,7.339793,0.043890,0.153375
21,FTNT,2020-06-25,0.429705,-0.206510,-0.401472,21.46,49.64,35.49,0,2.17,20.240546,0.009570,-0.020577
14,CLX,2020-06-25,0.769126,-0.174393,-0.411595,24.89,61.61,76.83,0,93.80,16.488216,0.016381,0.089078
7,NEWR,2020-06-25,0.683850,-0.163442,-0.404687,26.48,54.71,56.49,0,14.86,9.607704,0.038872,0.047525
26,W,2020-06-25,0.764687,-0.137101,-0.466114,27.88,62.94,75.02,0,46.36,59.678106,0.038528,0.277771
25,DDOG,2020-06-25,0.878251,-0.077674,-0.215103,28.57,75.35,91.11,0,65.46,18.094215,0.071599,0.262958


In [248]:
res.sort_values(by='ret1wk_mean',ascending=True)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret
17,DKNG,2020-06-25,0.453561,-0.390069,-0.274916,7.28,47.14,12.39,0,0.00,10.449977,-0.092349,0.047469
19,THO,2020-06-25,0.765212,-0.134022,-0.472665,30.98,61.97,73.98,0,0.83,24.261836,-0.024568,0.277242
9,MBUU,2020-06-25,0.397761,-0.294905,-0.429637,17.28,51.13,45.37,0,2.42,10.871005,-0.024467,0.040916
12,TSLA,2020-06-25,0.782697,-0.116140,-0.451458,32.09,57.16,62.43,0,1.23,168.163978,-0.012449,0.174466
0,SPY,2020-06-25,0.386321,-0.325179,-0.453047,14.86,48.41,31.22,0,0.83,24.742986,-0.006729,0.016863
21,FTNT,2020-06-25,0.429705,-0.206510,-0.401472,21.46,49.64,35.49,0,2.17,20.240546,0.009570,-0.020577
11,QQQ,2020-06-25,0.782800,-0.116203,-0.457449,30.10,59.19,63.86,0,6.70,20.964915,0.015232,0.063518
14,CLX,2020-06-25,0.769126,-0.174393,-0.411595,24.89,61.61,76.83,0,93.80,16.488216,0.016381,0.089078
8,DKS,2020-06-25,0.840570,-0.071497,-0.461413,38.08,61.38,73.92,0,18.46,7.338442,0.021941,0.172999
4,EA,2020-06-25,0.864977,-0.086052,-0.463726,37.87,66.37,87.95,0,92.85,10.819615,0.023074,0.116427


In [251]:
res[(res.RSI_percentile>90) & (res.k_change>-100)].sort_values(by='ret1wk_mean',ascending=False)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret
24,SNAP,2020-06-25,0.970298,-0.007973,-0.499707,50.12,77.16,97.20,1,89.35,3.866538,0.106816,0.381424
6,SHOP,2020-06-25,0.904691,-0.056009,-0.395170,35.53,73.10,90.87,0,98.75,170.848368,0.100288,0.180091
22,WK,2020-06-25,0.891602,-0.093390,-0.402602,31.82,78.05,96.30,0,98.35,6.661073,0.098848,0.265089
25,DDOG,2020-06-25,0.878251,-0.077674,-0.215103,28.57,75.35,91.11,0,65.46,18.094215,0.071599,0.262958
23,NET,2020-06-25,0.770322,-0.123808,-0.323673,30.23,68.16,91.85,0,95.96,4.606402,0.069942,0.293610
2,SVMK,2020-06-25,0.961295,0.047474,-0.427500,55.31,73.61,94.37,1,48.64,3.666484,0.054541,0.172156


In [ ]:
pk_444bcc6ef4a94b859405ef30fe66fa14 
https://cloud.iexapis.com/stable/stock/QQQ/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=1
    https://cloud.iexapis.com/stable/stock/CWH/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=30&chartIEXWhenNull=True

In [170]:
if ~(pd.to_datetime(list(response.json()['Time Series (Daily)'].keys())[0])==pd.Timestamp.today()):
    d={'function': 'TIME_SERIES_INTRADAY','apikey':"YEUC2DL0F7KC43PB",'symbol':'MBUU','interval':'1min'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )

In [189]:
d={'token':"pk_9469e57bc642462d905e4dc0003523ab",'chartIEXOnly':'True','chartLast':'1'}
response2 = requests.get(
        'https://cloud.iexapis.com/stable/stock/{}/intraday-prices?'.format('MBUU')#token={}&chartIEXOnly=True&chartLast=1'.format(holdings[0],'pk_9469e57bc642462d905e4dc0003523ab'),
        ,params=d,
        )

In [180]:
t=pd.DataFrame(response.json()[0]['date'],response.json()[0]['open'],response.json()[0]['high'],response.json()[0]['low'],response.json()[0]['close'],response.json()[0]['close'])

TypeError: __init__() takes from 1 to 6 positional arguments but 7 were given

In [190]:
date=list(response.json()['Time Series (Daily)'].keys())

In [192]:
date.insert(0,response2.json()[0]['date'])

In [193]:
date

['2020-05-20',
 '2020-05-19',
 '2020-05-18',
 '2020-05-15',
 '2020-05-14',
 '2020-05-13',
 '2020-05-12',
 '2020-05-11',
 '2020-05-08',
 '2020-05-07',
 '2020-05-06',
 '2020-05-05',
 '2020-05-04',
 '2020-05-01',
 '2020-04-30',
 '2020-04-29',
 '2020-04-28',
 '2020-04-27',
 '2020-04-24',
 '2020-04-23',
 '2020-04-22',
 '2020-04-21',
 '2020-04-20',
 '2020-04-17',
 '2020-04-16',
 '2020-04-15',
 '2020-04-14',
 '2020-04-13',
 '2020-04-09',
 '2020-04-08',
 '2020-04-07',
 '2020-04-06',
 '2020-04-03',
 '2020-04-02',
 '2020-04-01',
 '2020-03-31',
 '2020-03-30',
 '2020-03-27',
 '2020-03-26',
 '2020-03-25',
 '2020-03-24',
 '2020-03-23',
 '2020-03-20',
 '2020-03-19',
 '2020-03-18',
 '2020-03-17',
 '2020-03-16',
 '2020-03-13',
 '2020-03-12',
 '2020-03-11',
 '2020-03-10',
 '2020-03-09',
 '2020-03-06',
 '2020-03-05',
 '2020-03-04',
 '2020-03-03',
 '2020-03-02',
 '2020-02-28',
 '2020-02-27',
 '2020-02-26',
 '2020-02-25',
 '2020-02-24',
 '2020-02-21',
 '2020-02-20',
 '2020-02-19',
 '2020-02-18',
 '2020-02-

In [211]:
spy


,Date,open,high,low,close,adj_close,highest_high,lowest_low,k,k_change
5032,2000-05-18,89.00,89.36,85.38,85.75,75.0926,NaN,NaN,NaN,NaN
5031,2000-05-19,83.75,85.00,81.31,81.69,71.5372,NaN,NaN,NaN,NaN
5030,2000-05-22,81.63,82.25,75.58,81.63,71.4846,NaN,NaN,NaN,NaN
5029,2000-05-23,81.38,82.63,74.25,74.81,65.5123,NaN,NaN,NaN,NaN
5028,2000-05-24,75.00,79.88,72.25,79.50,69.6194,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4,2020-05-14,218.12,221.87,215.99,221.83,221.8300,228.095,203.63,0.727483,-0.253806
3,2020-05-15,218.77,223.33,218.33,223.27,223.2700,228.095,203.63,0.729614,-0.205561
2,2020-05-18,226.28,228.68,225.47,227.43,227.4300,228.680,203.63,0.832266,0.039379
1,2020-05-19,227.66,229.85,226.71,226.86,226.8600,229.850,203.63,0.879615,0.209120


In [93]:
np.nanpercentile(spy.k_change,q=10)

-0.45913701443037197

In [174]:
response.json()[0]

{'date': '2020-05-20',
 'minute': '12:30',
 'label': '12:30 PM',
 'high': None,
 'low': None,
 'open': None,
 'close': None,
 'average': None,
 'volume': 0,
 'notional': 0,
 'numberOfTrades': 0}

In [198]:
pd.to_datetime('today')

Timestamp('2020-05-20 12:48:48.505275')

In [200]:
pd.Timestamp.today().date()

datetime.date(2020, 5, 20)

In [201]:
pd.to_datetime(t[0]).date()

datetime.date(2020, 5, 19)

In [203]:
pd.to_datetime(t[0]).date()==pd.Timestamp.today().date()

False

In [272]:
get_intraday('BL')

[78.24, 78.35, 78.24, 78.3, 78.3]

In [173]:
spy.adj_close.rolling(90).std().values[-1]

57.6823412305509

In [175]:
(spy.adj_close/spy.adj_close.shift(22)-1).values[-1]

0.2918792647691646

In [178]:
(spy.adj_close/spy.adj_close.shift(5)-1).rolling(4).mean().values[-1]

0.11488104024483439